## Exercise 1

Create a vectorized version of log and exp math function for 1D array A = [2, 5, 10, 3, 8]

Results should be: 
+ [0.6931472 1.609438  2.3025851 1.0986123 2.0794415]
+ [7.3890562e+00 1.4841316e+02 2.2026465e+04 2.0085537e+01 2.9809580e+03]

In [4]:
import math as m
from numba import vectorize, float32, int32

@vectorize([float32(int32)], target='parallel', fastmath=True) 
def vec_log(x):
    return m.log(x)

@vectorize([float32(int32)], target='parallel', fastmath=True) 
def vec_exp(x):
    return m.exp(x)

array =  [2, 5, 10, 3, 8]

print(vec_log(array))
print(vec_exp(array))

[0.6931472 1.609438  2.3025851 1.0986123 2.0794415]
[7.3890562e+00 1.4841316e+02 2.2026465e+04 2.0085537e+01 2.9809580e+03]


## Exerice 2
Compute the value of a Gaussian probability density function at $x$ with $mean = 1$, $\sigma = 1$, lower and upper bound in $(-3, 3)$ and $size = 100000$

In [25]:
import matplotlib.pyplot as plt
import numpy as np

array = np.arange(-3, 3, 6e-5, dtype='float32')

@vectorize([float32[:](float32[:])]) 
def vec_exp(x):
    return np.exp((-1/2)*(x-1)**2)/(2*m.pi)
    
gaussValues = vec_exp(array)

plt.plot(gaussValues)

<ipython-input-25-9d1bedeb86e3>:6: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "vec_exp" failed type inference due to: No conversion from array(float64, 1d, C) to array(float32, 1d, A) for '$32return_value.15', defined at None

File "<ipython-input-25-9d1bedeb86e3>", line 8:
def vec_exp(x):
    return np.exp((-1/2)*(x-1)**2)/(2*m.pi)
    ^

During: typing of assignment at <ipython-input-25-9d1bedeb86e3> (8)

File "<ipython-input-25-9d1bedeb86e3>", line 8:
def vec_exp(x):
    return np.exp((-1/2)*(x-1)**2)/(2*m.pi)
    ^

  @vectorize([float32[:](float32[:])])
c:\users\141661\appdata\local\programs\python\python38\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "vec_exp" was compiled in object mode without forceobj=True.

File "<ipython-input-25-9d1bedeb86e3>", line 7:
@vectorize([float32[:](float32[:])]) 
def vec_exp(x):
^

  warnings.warn(errors.NumbaWarning(warn_msg,
c:\users\141661\appdata\loca

NotImplementedError: array(float32, 1d, A) cannot be represented as a Numpy dtype

## Exercise 3

Create a "zero suppression" function. A common operation when working with waveforms is to force all samples values below a certain absolute magnitude to be zero, as a way to eliminate low amplitude noise. 
Plot the data before and after the application of the zero_suppress function.

$thresold = 15$

In [13]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from numba import jit


n = 100000
noise = np.random.normal(size=n) * 3
pulses = np.maximum(np.sin(np.arange(n) / (n / 23)) - 0.3, 0.0)
data = ((pulses * 300) + noise).astype(np.int16)

#put your code here

@jit("int16[:](int16[:])",nopython=True)
def zero_suppression(X):
    n = len(X)
    Y = np.zeros(n)
    for i in range(n):
        if X[i]>=15:
            Y[i] = X[i]
            
suppressed_data = zero_suppression(data)

print(data)

#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,8))

#ax1.plot(data)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo conversion from none to array(int16, 1d, A) for '$60return_value.1', defined at None
[1m
File "<ipython-input-13-c4aaf5caeb37>", line 20:[0m
[1mdef zero_suppression(X):
    <source elided>
        if X[i]>=15:
[1m            Y[i] = X[i]
[0m            [1m^[0m[0m
[0m
[0m[1mDuring: typing of assignment at <ipython-input-13-c4aaf5caeb37> (20)[0m
[1m
File "<ipython-input-13-c4aaf5caeb37>", line 20:[0m
[1mdef zero_suppression(X):
    <source elided>
        if X[i]>=15:
[1m            Y[i] = X[i]
[0m            [1m^[0m[0m


## Exercise 4

Calculate the Sigmoid kernel between the matrix X and Y defined as below. The Sigmoid kernel is defined as:

$k(x,y) = \tanh(\alpha x^T y + c) $

In [4]:
X = np.random.rand(3,3)
Y = np.random.rand(3,3)

alpha = 1
c = 0
Z =

@jit("void(flat32[:,:], flat32[:,:], flat32[:,:])",nopython=True)
def SigKernel (X, Y, Z):
    Z = np.dot(X.T, Y)
    n, m = size(Z)
    for i in range(n):
        for j in range(m):
            Z[i, j] = m.tanh(alpha*Z[i,j]+c)

print(SigKernel(X,Y))

array([[0.63950462, 0.53306905, 0.73921925],
       [0.70600103, 0.11695335, 0.25826423],
       [0.81473669, 0.16700093, 0.16247506]])

## Exercise 5

Create a kernel function similar to the ```double_kernel``` see during the lecture and create a new function that takes a 3 dimensional matrix as input calculating the $cos$ for each element and then returns the result. The shape of the matrix must be $256X256X256$. The matrix can be randomly generated

## Exercise 6

Create a matrix multiplication kernel function, called ```matmul``` that takes as input two 2D matrices:
+ A of shape $24x12$
+ B of shape $12x22$
and that computes the multiplication and put the results into a third matrix C of shape $24x12$

A and B must be randomly generated and only int values are allowed.


In [ ]:
from __future__ import division
from numba import cuda
import numpy
import math

# complete the code
@cuda.jit
def matmul(A, B, C):
    C = np.dot(A, B)
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

# Initialize the data arrays
A = np.random.randint(10, size=(24, 12))
B = np.random.randint(10, size=(12, 22))

# Configure the blocks
threadsperblock = 
blockspergrid_x = 
blockspergrid_y = 
blockspergrid = 

